**PROJECT DESCRIPTION**

We would like to reproduce the research conducted by PhD students of the National University Ho Chi Minh. Research is focuing on Applying Fama and French Three Factors Model and Capital Asset Pricing Model in the Stock Exchange of Vietnam.

https://www.researchgate.net/publication/265152344_Applying_Fama_and_French_Three_Factors_Model_and_Capital_Asset_Pricing_Model_in_the_Stock_Exchange_of_Vietnam

**THEORETICAL BACKGROUND (SOURCE: CFA CIRRICULUM, LEVEL 2, 2019)**

In capital market theory, taking on a higher degree of unsystematic risk will not be compensated with a higher return because unsystematic risk can be estimated, without additional cost, through diversification. Only if an investor takes on a higher level of risk that cannot be easily diversified away should she expect to be rewarded in the form of higher return. Systemic risk is measured as the contribution of a security to the risk of a well-diversified portfolio. 
This conclusion has very important implications for asset pricing and expected returns. If risk is measured in terms of the standard deviation of returns of stock, the riskiest stock will not necessarily have the highest expected return.

In the CAPM, as asset’s non-diversifiable or systematic risk is positively related to its beta. Further, the CAPM asserts that differences in expected returns across assets can be explained by a single factor, the return on the market portfolio. The higher the assets’ beta risk (or sensitivity to the market), the higher its expected return.
CAPM is a single-index model that is widely used to estimate returns given security betas. According to CAPM, required return on stock is equal risk free-rate added to excess of market return which is scaled by beta estimate.

$\Large R_{i} = R_{f}+\beta_{i,mkt}(R_{m}-R_{f}) $

Differences in expected returns on different assets usually cannot be explained by just single market factor. This limitation is partially address by multifactor models like Fama and French models, Pastor-Stambaugh model, Carhart model etc.

The FF3 model attempts to account for other factors that are missed in CAPM.

$\Large R_{i} = R_{f}+\beta_{i,mkt}(R_{m}-R_{f})+\beta_{i,SMB}(SMB)+\beta_{i,HML}(HML) $

The FF3 model considers the following factors:
-	Equity risk premium. The FF3 model shares this factor with CAPM. The equity risk premium is calculated as the difference between the return on a value-weighted market index and the risk-free rate. The baseline value for the market beta is 1.
-	Small cap return premium (SMB). This factor accounts for differences in company market capitalizations. It is estimated as the difference between average return on three small-cap portfolios and the average return on three large-cap portfolios. The baseline value for the size beta is zero. It is usually positive for small-cap stocks. The smaller the company, the greater the required return.
-	Value return premium (HML). This factor accounts for differences in returns on value stocks and growth stocks. It is estimated as the difference between the average return on two high book-to-market portfolios (that represent a value bias) and the average return on two low book-to-market portfolios (that represent a growth bias). The baseline value for value of beta is zero. It is usually positive for stocks with high book-to-market ratios. The higher the ratio of book-to-market, the greater the required return.


**PORTFOLIOS AND FACTORS CREATION**


Most of publicly available databases don’t allow for the bulk download of necessary data. Therefore, the solution was to develop web-scrapping algorithm that would allow to extract series for all companies that were or are listed on Warsaw stock exchange in one attempt. Following web sources were sued for data extraction: www.bankier.pl, www.stooq.pl. Scrapped series are as of quarterly frequency and span from Q2 1991 till Q1 2022. The final dataset includes 273 polish companies. 

First, stocks are sorted every quarter by market value size and split into small (S) and big (B) groups, where S group contains stocks with market value below mean value of the population and B group contains stocks with market value above mean value of the population.
S and B values are assigned to each observation every quarter as per below:

In [1]:
import os
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.tsa.stattools import adfuller
import numpy

os.chdir('C:\\Users\\zymin\\Downloads')
cwd = os.getcwd()

In [2]:
df = pd.read_csv('final_df_short_ret_clean.csv')
df.set_index(['period'], drop=True, append=False, inplace=True, verify_integrity=False)
df.index = pd.to_datetime(df.index, format = '%d/%m/%Y')
df = df.sort_values(by = ['period', 'Market_value'], ascending = [False, True])
rf = pd.read_csv('rf.csv')
rf.set_index(['period'], drop=True, append=False, inplace=True, verify_integrity=False)
rf.index = pd.to_datetime(rf.index, format = '%d/%m/%Y')

In [3]:
df['market_value_SB'] = pd.qcut(df['Market_value'], q=2, labels = ['S', 'B'])
df.head()

,Ticker,close,perc_change,close_wig,perc_change_wig,Assets,Number of shares,Market_value,market_value_SB
period,,,,,,,,,
2021-12-31,SHD,0.41,-0.16,69296.26,-0.01,25412000,"27,530,111",11287345.51,S
2021-12-31,BCM,6.40,-0.20,69296.26,-0.01,45942000,"2,020,000",12928000.00,S
2021-12-31,CPL,6.20,-0.09,69296.26,-0.01,37868000,"3,539,547",21945191.40,S
2021-12-31,PAT,1.10,-0.30,69296.26,-0.01,136263000,"29,500,000",32450000.00,S
2021-12-31,IPO,1.10,-0.11,69296.26,-0.01,124246000,"34,130,700",37543770.00,S


In [4]:
df.dtypes

Ticker                object
close                float64
perc_change          float64
close_wig            float64
perc_change_wig      float64
Assets                 int64
Number of shares      object
Market_value         float64
market_value_SB     category
dtype: object

In [5]:
df['Assets']  = df['Assets'].astype(float)
df.dtypes

Ticker                object
close                float64
perc_change          float64
close_wig            float64
perc_change_wig      float64
Assets               float64
Number of shares      object
Market_value         float64
market_value_SB     category
dtype: object

Subsequently book to market ratio must be calculated for each observation with each B and S groups. This is done by just dividing assets column by market value column. Once book to market ratio is obtained stocks are again sorted within B and S groups and then consequently split and grouped into three subgroups based on book to market ration. Stocks that compose group with 30% highest book to market ratio are assigned with value H, stocks that compose group with 30% lowest book to market ratio are assigned with value L, and those that are within the middle 30% group are assigned with value M. Using such approach every quarter stocks are divided in S and B groups, and then within each S and B groups they are subdivided into 3 subgroups, namely H, M, L, based on their book to market ratio. As a result, every quarter we obtain 6 portfolios of stocks with each observation having an assigned value of S or B and H, M or L as per below:

In [6]:
df['B/M'] = df['Assets']/df['Market_value']
df.head()

,Ticker,close,perc_change,close_wig,perc_change_wig,Assets,Number of shares,Market_value,market_value_SB,B/M
period,,,,,,,,,,
2021-12-31,SHD,0.41,-0.16,69296.26,-0.01,25412000.0,"27,530,111",11287345.51,S,2.251371
2021-12-31,BCM,6.40,-0.20,69296.26,-0.01,45942000.0,"2,020,000",12928000.00,S,3.553682
2021-12-31,CPL,6.20,-0.09,69296.26,-0.01,37868000.0,"3,539,547",21945191.40,S,1.725572
2021-12-31,PAT,1.10,-0.30,69296.26,-0.01,136263000.0,"29,500,000",32450000.00,S,4.199168
2021-12-31,IPO,1.10,-0.11,69296.26,-0.01,124246000.0,"34,130,700",37543770.00,S,3.309364


In [7]:
df = df.sort_values(by = ['period', 'Market_value', 'B/M'], ascending = [True, True,True])
df.head()

,Ticker,close,perc_change,close_wig,perc_change_wig,Assets,Number of shares,Market_value,market_value_SB,B/M
period,,,,,,,,,,
1998-03-31,BRS,0.0826,-0.34,16864.0,0.15,61452000.0,"11,221,300",926879.380,S,66.299889
1998-03-31,EKP,0.7490,0.51,16864.0,0.15,15667000.0,"4,024,475",3014331.775,S,5.197504
1998-03-31,RLP,6.5000,-0.01,16864.0,0.15,72512000.0,976666,6348329.000,S,11.422218
1998-03-31,WIK,4.9700,-0.07,16864.0,0.15,23259000.0,"1,891,825",9402370.250,S,2.473738
1998-03-31,LBW,0.3470,-0.14,16864.0,0.15,25991000.0,"28,000,000",9716000.000,S,2.675072


In [8]:
df['B/M_LMH'] = pd.qcut(df['B/M'], q=3, labels = ['L', 'M','H'])
df.head()

,Ticker,close,perc_change,close_wig,perc_change_wig,Assets,Number of shares,Market_value,market_value_SB,B/M,B/M_LMH
period,,,,,,,,,,,
1998-03-31,BRS,0.0826,-0.34,16864.0,0.15,61452000.0,"11,221,300",926879.380,S,66.299889,H
1998-03-31,EKP,0.7490,0.51,16864.0,0.15,15667000.0,"4,024,475",3014331.775,S,5.197504,H
1998-03-31,RLP,6.5000,-0.01,16864.0,0.15,72512000.0,976666,6348329.000,S,11.422218,H
1998-03-31,WIK,4.9700,-0.07,16864.0,0.15,23259000.0,"1,891,825",9402370.250,S,2.473738,M
1998-03-31,LBW,0.3470,-0.14,16864.0,0.15,25991000.0,"28,000,000",9716000.000,S,2.675072,H


Using portfolios of stock created we calculate average rate of retrun to obtain quarterly return series for each portfolio.

In [9]:
df_SL = df['perc_change'].loc[(df['market_value_SB'] == 'S')&(df['B/M_LMH'] == 'L')]
df_SL = pd.DataFrame(data=df_SL)
df_SL = df_SL.groupby(df_SL.index.date).mean()
df_SM = df['perc_change'].loc[(df['market_value_SB'] == 'S')&(df['B/M_LMH'] == 'M')]
df_SM = pd.DataFrame(data=df_SM)
df_SM = df_SM.groupby(df_SM.index.date).mean()
df_SH = df['perc_change'].loc[(df['market_value_SB'] == 'S')&(df['B/M_LMH'] == 'H')]
df_SH = pd.DataFrame(data=df_SH)
df_SH = df_SH.groupby(df_SH.index.date).mean()
df_BH = df['perc_change'].loc[(df['market_value_SB'] == 'B')&(df['B/M_LMH'] == 'H')]
df_BH = pd.DataFrame(data=df_BH)
df_BH = df_BH.groupby(df_BH.index.date).mean()
df_BM = df['perc_change'].loc[(df['market_value_SB'] == 'B')&(df['B/M_LMH'] == 'M')]
df_BM = pd.DataFrame(data=df_BM)
df_BM = df_BM.groupby(df_BM.index.date).mean()
df_BL = df['perc_change'].loc[(df['market_value_SB'] == 'B')&(df['B/M_LMH'] == 'L')]
df_BL = pd.DataFrame(data=df_BL)
df_BL = df_BL.groupby(df_BL.index.date).mean()
df_wig = df['perc_change_wig']
df_wig = pd.DataFrame(data=df_wig)
df_wig.rename(columns = {'perc_change_wig':'perc_change'}, inplace = True)
df_wig = df_wig.groupby(df_wig.index.date).mean()

In [10]:
merged_df_portf = pd.concat([df_SL, df_SM, df_SH, df_BH, df_BM, df_BL,df_wig], axis = 1)
merged_df_portf.columns = ['SL','SM', 'SH', 'BH', 'BM', 'BL', 'wig']
merged_df_portf.head()

,SL,SM,SH,BH,BM,BL,wig
1998-03-31,-0.035000,0.027000,0.057500,-0.070,0.033333,0.088571,0.15
1998-06-30,-0.256667,-0.244545,-0.349333,0.090,-0.096667,-0.134000,-0.07
1998-09-30,-0.510000,-0.080000,-0.396364,-0.210,-0.520000,-0.197500,-0.22
1998-12-31,0.180000,0.271667,0.030800,0.090,0.075000,0.325000,0.05
1999-03-31,0.068000,-0.002500,0.010312,0.225,0.008333,0.176667,0.10


Using standard methodology proposed orinally by Fama and French also used in the research we are trying to reproduce we create following factors

$\Large SMB = 1/3(SH+SM+SL)-1/3(BH+BM+BL) $

$\Large HML = 1/2(SH+BH)-1/2(SL+BL) $

In [11]:
SMB = 1/3*(df_SH+df_SM+df_SL)-1/3*(df_BH+df_BM+df_BL)
HML = 0.5*(df_SH+df_BH)-0.5*(df_SL+df_BL)

The idea behind CAPM and FF3 is to estimate return in excess of risk-free rate of return, therefore we have to subtract risk-free rate of return from each of our dependent variable and also from market factor. We use German 10-year yield on government bond as a risk-free rate. Germany is perceived as the strongest economy within European Union with financial markets being the most developed ones, least volatile and one of the safest among whole European continent. Therefore, theoretically investors across Europe can easily invest in German 10-year government bond and earn rate of return which theoretically incurs no risk. After stock portfolios and factors creation is completed, all necessary series are adjusted by risk-free rate we obtain following series which are final and can directly be used for regression analysis:

In [12]:
df_SL_rf = df_SL - rf
df_SM_rf = df_SM - rf
df_SH_rf = df_SH - rf
df_BH_rf = df_BH - rf
df_BM_rf = df_BM - rf
df_BL_rf = df_BL - rf
df_wig_rf = df_wig - rf

In [13]:
df_SL_rf.rename(columns = {'perc_change':'SL'}, inplace = True)
df_SM_rf.rename(columns = {'perc_change':'SM'}, inplace = True)
df_SH_rf.rename(columns = {'perc_change':'SH'}, inplace = True)
df_BH_rf.rename(columns = {'perc_change':'BH'}, inplace = True)
df_BM_rf.rename(columns = {'perc_change':'BM'}, inplace = True)
df_BL_rf.rename(columns = {'perc_change':'BL'}, inplace = True)
rf.rename(columns = {'perc_change':'rf'}, inplace = True)
HML.rename(columns = {'perc_change':'HML'}, inplace = True)
SMB.rename(columns = {'perc_change':'SMB'}, inplace = True)
df_wig_rf.rename(columns = {'perc_change':'wig'}, inplace = True)

In [14]:
merged_df = pd.concat([df_SL_rf, df_SM_rf, df_SH_rf, df_BH_rf, df_BM_rf, df_BL_rf, HML, SMB, df_wig_rf], axis = 1)
merged_df.dropna(axis=0, inplace = True)
merged_df.head()

,SL,SM,SH,BH,BM,BL,HML,SMB,wig
1998-03-31,-0.083884,-0.021884,0.008616,-0.118884,-0.015551,0.039687,-0.033036,-0.000802,0.101116
1998-06-30,-0.300570,-0.288449,-0.393236,0.046097,-0.140570,-0.177903,0.065667,-0.236626,-0.113903
1998-09-30,-0.550162,-0.120162,-0.436525,-0.250162,-0.560162,-0.237662,0.050568,-0.019621,-0.260162
1998-12-31,0.141371,0.233037,-0.007829,0.051371,0.036371,0.286371,-0.192100,-0.002511,0.011371
1999-03-31,0.027258,-0.043242,-0.030429,0.184258,-0.032409,0.135925,-0.004677,-0.111396,0.059258


In [15]:
merged_df.describe()

,SL,SM,SH,BH,BM,BL,HML,SMB,wig
count,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000
mean,0.073297,0.031093,-0.013056,-0.015920,0.015309,0.064057,-0.083165,0.009296,-0.003156
std,0.342338,0.163879,0.178613,0.159015,0.146441,0.179085,0.187989,0.126664,0.112280
min,-0.550162,-0.369277,-0.436525,-0.471838,-0.560162,-0.291249,-1.125308,-0.293333,-0.300727
25%,-0.081068,-0.060607,-0.098860,-0.098672,-0.046632,-0.024661,-0.141320,-0.054552,-0.070288
50%,0.028538,0.011184,-0.027076,-0.038858,0.003379,0.041964,-0.059100,-0.011299,0.013244
75%,0.151316,0.101646,0.092657,0.089151,0.099103,0.144877,0.003571,0.042632,0.072226
max,1.609352,0.495347,0.456320,0.451364,0.369087,0.663871,0.312500,0.534574,0.236979


**CHECKING STATIONARITY**

Before applying regression analysis to the series, it is necessary to make sure that series are stationary. To confirm stationarity of series we will apply Augmented Dickey-Fuller (ADF) test. The Null hypothesis of ADF test states that series are non-stationary. After applying ADF test to each of our series we obtain below results:

In [16]:
for (colname,colval) in merged_df.iteritems():
    result = adfuller(colval.values)
    print(colname, 'ADF p-value: %f' % result[1])

SL ADF p-value: 0.000000
SM ADF p-value: 0.000037
SH ADF p-value: 0.000126
BH ADF p-value: 0.000000
BM ADF p-value: 0.000000
BL ADF p-value: 0.000000
HML ADF p-value: 0.000000
SMB ADF p-value: 0.000000
wig ADF p-value: 0.000000


From the above it can be clearly concluded that Null hypothesis can be rejected for each series that is analysed and therefore, we can state that all series are stationary. Now, for each portfolio of stocks that was created we proceed with application of CAPM and FF3 and compare the results of the regression estimation. The method used for regression analysis is simple Ordinary Least Squared regression.

**RUNNING CAPM AND FF3 REGRESSIONS**

First, we apply CAPM model to SL portfolio which includes companies that are small in size and have low book to market stock ratio. Following results were obtained:

In [17]:
model_CAPM_SL = smf.ols(formula = 'SL ~ wig', data = merged_df).fit()
print(model_CAPM_SL.summary())

                            OLS Regression Results                            
Dep. Variable:                     SL   R-squared:                       0.151
Model:                            OLS   Adj. R-squared:                  0.142
Method:                 Least Squares   F-statistic:                     16.39
Date:                Wed, 01 Jun 2022   Prob (F-statistic):           0.000107
Time:                        17:49:19   Log-Likelihood:                -24.407
No. Observations:                  94   AIC:                             52.81
Df Residuals:                      92   BIC:                             57.90
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0770      0.033      2.355      0.0

Market factor, which is presented by series called ‘wig’, is the only independent variable in CAPM model. As it can be seen market factor is statistically significant at 99% confidence level with p-value being equal to 0.000. Market factor can explain 14.2% of variation in SL portfolio returns as per adj. R-squared estimate. Intercept estimate is also statistically significant at 95% confidence level. From those results we can see that market factor alone can explain very small portion of variation in SL portfolio returns as is confirmed by small adj. R-squared estimate and significant intercept term. Next, we keep the same dependent variable which is returns on SL portfolio and apply FF3 model. Following results were obtained:

In [18]:
model_FF_SL = smf.ols(formula = 'SL ~ wig + HML + SMB', data = merged_df).fit()
print(model_FF_SL.summary())

                            OLS Regression Results                            
Dep. Variable:                     SL   R-squared:                       0.916
Model:                            OLS   Adj. R-squared:                  0.913
Method:                 Least Squares   F-statistic:                     327.1
Date:                Wed, 01 Jun 2022   Prob (F-statistic):           2.87e-48
Time:                        17:49:20   Log-Likelihood:                 84.301
No. Observations:                  94   AIC:                            -160.6
Df Residuals:                      90   BIC:                            -150.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0036      0.012     -0.298      0.7

As a reminder for FF3 model we add size (SMB) and value (HML) factors. First drastic difference is an improvement in adj. R-squared estimate. It increases from 14.2% to 91.3%. in another words FF3 model can explain 91.3% of SL portfolio variation of returns. Another improvement is that intercept becomes insignificant which means that most of variation in returns is explained purely by FF3 model factors. All independent variables are statistically significant at 99% level of confidence. SMB variable has positive coefficient confirms statement that SL portfolio contains small size companies. HML variable is negative which confirms that SL portfolio contains low book to value companies. The conclusion can be made that FF3 model significantly outperforms CAPM model when applied to SL portfolio.
	Next, we apply CAPM model to SM portfolio which includes companies that are small in size and have middle range book to market ratio. Following results were obtained:


In [19]:
model_CAPM_SM = smf.ols(formula = 'SM ~ wig', data = merged_df).fit()
print(model_CAPM_SM.summary())

                            OLS Regression Results                            
Dep. Variable:                     SM   R-squared:                       0.505
Model:                            OLS   Adj. R-squared:                  0.499
Method:                 Least Squares   F-statistic:                     93.81
Date:                Wed, 01 Jun 2022   Prob (F-statistic):           1.05e-15
Time:                        17:49:20   Log-Likelihood:                 70.171
No. Observations:                  94   AIC:                            -136.3
Df Residuals:                      92   BIC:                            -131.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0344      0.012      2.873      0.0

As can be seen, both market factor and intercept term are statistically significant 99% confidence level. Adj. R-squared stays at almost 50% which means that CAPM model can explain almost half of the variation of returns of SM portfolio. Now FF3 model is applied and following results were obtained:

In [20]:
model_FF_SM = smf.ols(formula = 'SM ~ wig + HML + SMB', data = merged_df).fit()
print(model_FF_SM.summary())

                            OLS Regression Results                            
Dep. Variable:                     SM   R-squared:                       0.619
Model:                            OLS   Adj. R-squared:                  0.606
Method:                 Least Squares   F-statistic:                     48.66
Date:                Wed, 01 Jun 2022   Prob (F-statistic):           8.75e-19
Time:                        17:49:20   Log-Likelihood:                 82.440
No. Observations:                  94   AIC:                            -156.9
Df Residuals:                      90   BIC:                            -146.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0503      0.012      4.069      0.0

We can observe that all the variables as well as intercept term are significant at 99% confidence level. However, given that intercept term stayed significant we can assume that FF3 factors don’t fully capture the variation of returns of SM portfolio. This also can be seen by relatively small improvement of adj. R-squared when comparing to the one obtained from CAPM model. Adj. R-squared improved by around 10% and reached 60.6% for FF3 model. Therefore, we can conclude that FF3 does add more value when compared to CAPM, however improvements are relatively marginal.
	The last from group of portfolios that are small in size is SH portfolio which contains companies with high book to market stock ratio. When applying CAPM to SH portfolio we obtain following results:


In [21]:
model_CAPM_SH = smf.ols(formula = 'SH ~ wig', data = merged_df).fit()
print(model_CAPM_SH.summary())

                            OLS Regression Results                            
Dep. Variable:                     SH   R-squared:                       0.578
Model:                            OLS   Adj. R-squared:                  0.574
Method:                 Least Squares   F-statistic:                     126.2
Date:                Wed, 01 Jun 2022   Prob (F-statistic):           6.04e-19
Time:                        17:49:21   Log-Likelihood:                 69.633
No. Observations:                  94   AIC:                            -135.3
Df Residuals:                      92   BIC:                            -130.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0092      0.012     -0.768      0.4

Above results show that market factor is significant at 99% confidence level. On the other hand intercept term is insignificant. Overall CAPM model can explain 57.4% of SH portfolio variation of returns as can confirmed by adj. R-squared estimate. Next, we apply FF3 model to SH portfolio and obtain following results:

In [22]:
model_FF_SH = smf.ols(formula = 'SH ~ wig + HML + SMB', data = merged_df).fit()
print(model_FF_SH.summary())

                            OLS Regression Results                            
Dep. Variable:                     SH   R-squared:                       0.736
Model:                            OLS   Adj. R-squared:                  0.727
Method:                 Least Squares   F-statistic:                     83.64
Date:                Wed, 01 Jun 2022   Prob (F-statistic):           6.15e-26
Time:                        17:49:21   Log-Likelihood:                 91.638
No. Observations:                  94   AIC:                            -175.3
Df Residuals:                      90   BIC:                            -165.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0035      0.011      0.309      0.7

It can been noted that all 3 factors stay significant at 99% confidence level. Same as for CAPM intercept term stay insignificant. Adj. R-squared estimate increased by approximately 15% and reached 72.7% for FF3 model. Positive coefficient for SMB factor confirms the fact that SH portfolio contains small size companies. Also, positive coefficient for HML factor confirms that SH portfolio contains companies that have high book to market stock ratio. The conclusion can be made that FF3 model adds value when compared to CAPM for SH portfolio.  
	Now we move to estimation of CAPM and FF3 models for portfolios that include companies that big in size, namely focus will on following portfolios: BH, BM and BL.
	First. CAPM model will be applied for BH portfolio which contains companies that big in size and have high book to market stock ratio. After applying CAPM to BH portfolio, following results were obtained:


In [23]:
model_CAPM_BH = smf.ols(formula = 'BH ~ wig', data = merged_df).fit()
print(model_CAPM_BH.summary())

                            OLS Regression Results                            
Dep. Variable:                     BH   R-squared:                       0.582
Model:                            OLS   Adj. R-squared:                  0.578
Method:                 Least Squares   F-statistic:                     128.1
Date:                Wed, 01 Jun 2022   Prob (F-statistic):           4.02e-19
Time:                        17:49:22   Log-Likelihood:                 80.970
No. Observations:                  94   AIC:                            -157.9
Df Residuals:                      92   BIC:                            -152.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0125      0.011     -1.173      0.2

It can be observed that market factor is significant at 99% confidence level which intercept term being insignificant. Adj. R-squared of 57.8% means that more than half of SH portfolio variation of returns is explained by CAPM model. Further, following results for FF3 model were obtained:

In [24]:
model_FF_BH = smf.ols(formula = 'BH ~ wig + HML + SMB', data = merged_df).fit()
print(model_FF_BH.summary())

                            OLS Regression Results                            
Dep. Variable:                     BH   R-squared:                       0.660
Model:                            OLS   Adj. R-squared:                  0.649
Method:                 Least Squares   F-statistic:                     58.32
Date:                Wed, 01 Jun 2022   Prob (F-statistic):           4.93e-21
Time:                        17:49:22   Log-Likelihood:                 90.715
No. Observations:                  94   AIC:                            -173.4
Df Residuals:                      90   BIC:                            -163.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0114      0.011      1.003      0.3

Interesting observation would be the fact that SMB factor is insignificant. Therefore, it means that SMB factor adds no value to explanation of variation in BH portfolio returns. On the other hand, both market and HML factor shows significance at 99% confidence level. FF3 manged to explain 64.9% of variation in returns of BH portfolio of stocks which can be seen by adj. R-squared, which is only 7% more than what was obtained from CAPM model estimates. Therefore, the conclusion can be made that FF3 adds little value when comparing to CAPM.
Further we test CAPM model on BM portfolio. After running the regression we obtain following results:


In [25]:
model_CAPM_BM = smf.ols(formula = 'BM ~ wig', data = merged_df).fit()
print(model_CAPM_BM.summary())

                            OLS Regression Results                            
Dep. Variable:                     BM   R-squared:                       0.642
Model:                            OLS   Adj. R-squared:                  0.638
Method:                 Least Squares   F-statistic:                     164.7
Date:                Wed, 01 Jun 2022   Prob (F-statistic):           3.26e-22
Time:                        17:49:22   Log-Likelihood:                 95.937
No. Observations:                  94   AIC:                            -187.9
Df Residuals:                      92   BIC:                            -182.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0186      0.009      2.046      0.0

Market factor is significant at 99% confidence level. Intercept term is also significant at 95% confidence level. CAPM manged to explain 63.8% of variation in returns of BM portfolio which is confirmed by adj. R-squared estimate. Now, we can compare that to the results obtained from FF3 model. After applying FF3 model to BM portfolio following results are obtained:

In [26]:
model_FF_BM = smf.ols(formula = 'BM ~ wig + HML + SMB', data = merged_df).fit()
print(model_FF_BM.summary())

                            OLS Regression Results                            
Dep. Variable:                     BM   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.632
Method:                 Least Squares   F-statistic:                     54.31
Date:                Wed, 01 Jun 2022   Prob (F-statistic):           3.94e-20
Time:                        17:49:23   Log-Likelihood:                 96.275
No. Observations:                  94   AIC:                            -184.6
Df Residuals:                      90   BIC:                            -174.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0204      0.011      1.909      0.0

By looking at the results it can be stated that this an example when FF3 deteriorates the results when compared with CAPM model results. First, among all factors only market factor stays significant which means that SMB and HML add no value at all when it comes to explaining the variation of BM portfolio returns. Also, adj. R-squared estimates slightly decreased to 63.2%. Therefore, the conclusion can be made that FF3 adds no value in explaining variation of BM portfolio returns.
	Finally, CAPM and FF3 will be applied to BL portfolio. After running CAPM for BL portfolio following results were obtained:


In [27]:
model_CAPM_BL = smf.ols(formula = 'BL ~ wig', data = merged_df).fit()
print(model_CAPM_BL.summary())

                            OLS Regression Results                            
Dep. Variable:                     BL   R-squared:                       0.442
Model:                            OLS   Adj. R-squared:                  0.436
Method:                 Least Squares   F-statistic:                     72.84
Date:                Wed, 01 Jun 2022   Prob (F-statistic):           2.75e-13
Time:                        17:49:23   Log-Likelihood:                 56.202
No. Observations:                  94   AIC:                            -108.4
Df Residuals:                      92   BIC:                            -103.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0674      0.014      4.856      0.0

Both market factor and intercept term are significant at 99% confidence level. Adj. R-squared stands at 43.6%. After running FF3 on BL portfolio we get following results:

In [28]:
model_FF_BL = smf.ols(formula = 'BL ~ wig + HML + SMB', data = merged_df).fit()
print(model_FF_BL.summary())

                            OLS Regression Results                            
Dep. Variable:                     BL   R-squared:                       0.688
Model:                            OLS   Adj. R-squared:                  0.677
Method:                 Least Squares   F-statistic:                     66.04
Date:                Wed, 01 Jun 2022   Prob (F-statistic):           1.16e-22
Time:                        17:49:24   Log-Likelihood:                 83.479
No. Observations:                  94   AIC:                            -159.0
Df Residuals:                      90   BIC:                            -148.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0184      0.012      1.507      0.1

Important observation here would the fact that all three factors are significant at 99% confidence level. On the other hand, intercept term lost its significance. Adj. R-squared increased by more than 20% reaching 67.7% for FF3 model. The conclusion can be made that FF3 outperforms CAPM in capturing variation of BL portfolio returns. Negative coefficient for SMB factor confirms that BL portfolio contains companies that are big in size. Also, negative coefficient for HML factor confirms that BL portfolio contains companies that have low book to market stock ratio.

**POLISH MARKET RESULTS**

|      | **CAPM**  |  |        | **Fama and French** |  |   |   |          |  |
|---------------|-----------|-----------|----------------|---------------------|-----------|-----------|------------|-----------------|------|
| **Portfolio** | Intercept | Market    | Adj. R-squared | Intercept           | Market    | SMB       | HML        | Adj. R-squared  |      |
| **S/L**       | 0.08     | 1.19*** | 0.14          | -0.01             | 1.09*** | 1.31*** | -0.82*** | 0.91           |      |
| **S/M**       | 0.03**  | 1.04*** | 0.50         | 0.05***           | 0.98*** | 0.65*** | 0.27***  | 0.60           |      |
| **S/H**       | -0.01   | 1.21*** | 0.57          | 0.01              | 1.14*** | 0.78*** | 0.24**   | 0.72           |      |
| **B/L**       | 0.067*** | 1.06*** | 0.43          | 0.02              | 1.10*** | -0.37** | -0.63*** | 0.68          |      |
| **B/M**       | 0.019    | 1.05*** | 0.64          | 0.02              | 1.05*** | -0.04   | 0.02     | 0.63           |      |
| **B/H**       | -0.01   | 1.08*** | 0.58          | 0.01              | 1.06*** | 0.15    | 0.30***  | 0.65           |      |
         |           |           |                |                     |           |           |            |                 |      | 

**VIETNAMESE MARKET RESULTS**

|       | **CAPM**  |  |        | **Fama and French** |  |  |  |            |  |
|---------------|-----------|----------|----------------|---------------------|----------|----------|----------|-----------------|------|
| **Portfolio** | Intercept | Market   | Adj. R-squared | Intercept           | Market   | SMB      | HML      | Adj. R-squared  |      |
| **S/L**       | 3.34      | 1.26***  | 0.74         | 1.04                | 1.20***  | 0.62***  | -0.78*** | 0.83          |      |
| **S/M**       | 0.47      | 1.01***  | 0.74         | 0.82                | 1.05***  | 0.50***  | -0.08    | 0.78          |      |
| **S/H**       | -1.16    | 0.99***  | 0.69         | 1.40                | 1.15***  | 0.61***  | 0.56***  | 0.80          |      |
| **B/L**       | 4.27      | 1.30***  | 0.81         | 1.40                | 1.16***  | -0.47*** | -0.54*** | 0.87          |      |
| **B/M**       | 0.61      | 1.03***  | 0.76         | 0.78                | 1.02***  | -0.32**  | 0.15     | 0.77          |      |
| **B/H**       | 1.30       | 1.25***  | 0.80         | 1.06                | 1.21***  | -0.47*** | 0.09     | 0.81          |      |
|       |           |          |                |                     |             


**CONCLUSION**

Even though due to different data sets used for the original research and our project we couldn't fully numerically recreate the original findings, however, we obtained quite similar results from conceptual and statistical point of view. For both our results and original estimates FF3 model outperformed CAPM in terms of adjusted R-squared results for most of portfolios of stocks. Also, for both our results and original research FF3 produced the worst results for B/M portfolio with some factors being statistically insignificant and thus irrelevant for explanation of variation of portfolio stock returns. 